# Lab12

---

## Task

Для нахождения минимума сильно выпуклой функции (одной переменной) реализовать градиентный спуск (по одной из стратегий выбора шага) и метод Нестерова.

Потестировать на нескольких функциях; сравнить количества итераций.

# Solution

---

In [ ]:
import numpy as np
from scipy import misc

from plotly import graph_objects

### Реализация градиентного спуска

In [ ]:
def get_derivative(func, h):
    return lambda x: misc.derivative(func, x, dx=h)

def gradient_descent(func, x0, rate, rate_decrease, eps=10 ** (-8), max_iter=100):
    """
    Gradient descent using step fragmentation strategy
    """
    x = x0
    points = [x0]
    df = get_derivative(func, 1e-10)
    grad = df(x0)
    
    while abs(grad) > eps:
        grad = df(x)
        
        x = x - rate * grad
        points.append(x)
        
        if len(points) > max_iter:
            break
    
        rate = (1 - rate_decrease) * rate
    
    return x, points

### Метод Нестерова

In [ ]:
def nesterov_method(f, y0, jitter, eps=10**(-8), max_iter=100):
    x = y0
    y = y0
    a = 1
    df = get_derivative(f, 1e-10)
    z = y - jitter
    alpha = abs((y - z) / (df(y) - df(z)))
    points = [x]

    while abs(df(points[-1])) > eps:
        i = 0
        grad = df(y)
        
        while f(y) - f(y - 2 ** (-i) * alpha * grad) < 2 ** (- i - 1) * alpha * grad ** 2:
            i += 1
            if i >= 10**6:
                print(f'Jump overflow at iteration {len(points)}')
                return x, points

        alpha = 2 ** (-i) * alpha
        x = y - alpha * grad
        points.append(x)
        
        if len(points) > max_iter:
            break
        prev_a = a
        a = (1 + np.sqrt(4 * prev_a ** 2 + 1)) / 2
        y = x + (prev_a - 1) * (x - points[-2]) / a
    
    return x, points

## Experimental reseach

In [ ]:
def experiment(func, segment, rate, x0):
    a, b = segment
    grid = np.linspace(a, b, num=300)

    x_min, points = gradient_descent(func, x0, rate, rate_decrease=0.02)
    
    fig1 = graph_objects.Figure()
    fig1.add_trace(graph_objects.Scatter(x=points, y=[func(x) for x in points],mode='lines+markers'),)
    fig1.add_trace(graph_objects.Scatter(x=grid, y=[func(x) for x in grid], mode='lines'))

    fig1.update_layout(autosize=False, template="simple_white", showlegend=False)
    fig1.show()

    print("Gradient descent: step fragmentation")
    print(f"Found minimum: {x_min}")
    print(f"Iterations: {len(points)}")

    x_min, points = nesterov_method(func, y0=0, jitter=0.1)

    fig2 = graph_objects.Figure()
    fig2.add_trace(graph_objects.Scatter(x=points, y=[func(x) for x in points],mode='lines+markers'))
    fig2.add_trace(graph_objects.Scatter(x=grid, y=[func(x) for x in grid], mode='lines'))

    fig2.update_layout(autosize=False, template="simple_white", showlegend=False)
    fig2.show()

    print("Nesterov's method")
    print(f"Found minimum: {x_min}")
    print(f"Iterations: {len(points)}")


In [ ]:
f = lambda x: x * (x + 1) 
rate = 0.7
x0 = -2

experiment(f, (-10, 10), rate, x0)

In [ ]:
f = lambda x: 3 * np.sin(x)
rate = 0.4
x0 = 0

experiment(f, (-4, 4), rate, x0)

In [ ]:
f = lambda x: x * x * x - 5 * x
rate = 0.2
x0 = 0

experiment(f, (-2, 2), rate, x0)